In [119]:
from sklearn.neighbors import KNeighborsClassifier
import yfinance as yf
import pandas as pd 
import sklearn.model_selection as sk
from sklearn.metrics import accuracy_score
import numpy as np

In [120]:
data = yf.download(' aapl ',period="7D",interval="1m")
data=data.rename({"High":"high","Low":"low","Close":"close","Open":"open","Adj Close":"adj close","Volume":"volume"},axis=1)

data

[*********************100%%**********************]  1 of 1 completed


,open,high,low,close,adj close,volume
Datetime,,,,,,
2023-08-30 09:30:00-04:00,184.940002,185.355499,184.740005,185.320007,185.320007,1743766
2023-08-30 09:31:00-04:00,185.300003,185.559906,185.214600,185.542297,185.542297,450686
2023-08-30 09:32:00-04:00,185.550003,185.779999,185.375000,185.735001,185.735001,366096
2023-08-30 09:33:00-04:00,185.739105,185.750000,185.150101,185.220001,185.220001,381661
2023-08-30 09:34:00-04:00,185.214996,185.460007,185.070007,185.410004,185.410004,273354
...,...,...,...,...,...,...
2023-09-08 15:55:00-04:00,178.000000,178.139999,177.990005,178.130005,178.130005,376594
2023-09-08 15:56:00-04:00,178.134995,178.179993,178.029999,178.170197,178.170197,312656
2023-09-08 15:57:00-04:00,178.175003,178.220001,178.119995,178.160004,178.160004,315770


In [121]:
from copy import deepcopy as dc
def prepareforlstm(df,loookback):
    df=pd.DataFrame(df["close"])
    df=dc(df)
    for i in range(1,loookback+1):
        df[f'close(t-{i})']=df["close"].shift(i)
    df.dropna(inplace=True)
    return df

In [122]:
lookback=7

In [123]:
shifted_df=prepareforlstm(data,lookback)
shifted_df

,close,close(t-1),close(t-2),close(t-3),close(t-4),close(t-5),close(t-6),close(t-7)
Datetime,,,,,,,,
2023-08-30 09:37:00-04:00,184.919998,184.966095,185.070007,185.410004,185.220001,185.735001,185.542297,185.320007
2023-08-30 09:38:00-04:00,185.251495,184.919998,184.966095,185.070007,185.410004,185.220001,185.735001,185.542297
2023-08-30 09:39:00-04:00,185.524994,185.251495,184.919998,184.966095,185.070007,185.410004,185.220001,185.735001
2023-08-30 09:40:00-04:00,185.470001,185.524994,185.251495,184.919998,184.966095,185.070007,185.410004,185.220001
2023-08-30 09:41:00-04:00,185.490097,185.470001,185.524994,185.251495,184.919998,184.966095,185.070007,185.410004
...,...,...,...,...,...,...,...,...
2023-09-08 15:55:00-04:00,178.130005,177.994995,177.889999,177.820007,177.985001,177.940002,177.925003,178.000107
2023-09-08 15:56:00-04:00,178.170197,178.130005,177.994995,177.889999,177.820007,177.985001,177.940002,177.925003
2023-09-08 15:57:00-04:00,178.160004,178.170197,178.130005,177.994995,177.889999,177.820007,177.985001,177.940002


In [124]:
from sklearn.preprocessing import MinMaxScaler
scalar=MinMaxScaler(feature_range=(-1,1))
shifted_df_as_np=scalar.fit_transform(shifted_df)


In [125]:
x=shifted_df_as_np[:,1:]
y=shifted_df_as_np[:,0]
x.shape,y.shape

((2714, 7), (2714,))

In [126]:
split_index=int(len(x)*0.90)

In [127]:
x_train=x[:split_index].reshape((-1,lookback))
x_test=x[split_index:].reshape((-1,lookback))
y_train=y[:split_index].reshape((-1,1))
y_test=y[split_index:].reshape((-1,1))
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((2442, 7), (272, 7), (2442, 1), (272, 1))

In [128]:
knn=KNeighborsClassifier(n_neighbors=7)
knn.fit(X= x_train, y=y_train)
accuracy_train = accuracy_score(y_train, knn.predict(x_train))
accuracy_test = accuracy_score(y_test, knn.predict(x_test))

/home/batuhan-saylam/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


ValueError: Unknown label type: continuous. Maybe you are trying to fit a classifier, which expects discrete classes on a regression target with continuous values.